In [10]:
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 80.3 MB/s  0:00:006m0:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# ---------- video transform ----------
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def read_video(path, n_frames=16):
    cap = cv2.VideoCapture(path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    idxs = np.linspace(0, max(length - 1, 0), n_frames).astype(int)

    frames = []
    i = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if i in idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(transform(frame))
        i += 1

    cap.release()

    # pad if video shorter than n_frames
    while len(frames) < n_frames:
        frames.append(frames[-1])

    return torch.stack(frames, dim=1)  # (3, T, 224, 224)


# ---------- Dataset ----------
class EchoNetA4C(Dataset):
    def __init__(self, root, split="train", n_frames=16):
        self.root = root
        self.n_frames = n_frames

        df = pd.read_csv(os.path.join(root, "FileList.csv"))

        if split == "train":
            df = df[df["Split"] != 5]
        else:
            df = df[df["Split"] == 5]

        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        video_path = os.path.join(self.root, "Videos", row["FileName"])

        x = read_video(video_path, self.n_frames)
        y = torch.tensor(row["EF"], dtype=torch.float32)

        return x, y

In [4]:
train_ds = EchoNetA4C(
    "/home/users/joshua04/141/data/raw/echonet_pediatric/A4C",
    split="train",
    n_frames=16
)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

In [2]:
import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

torch: 2.5.1+cu121
cuda available: True
GPU: NVIDIA L40S


In [3]:
import os

print("Current working directory:")
print(os.getcwd())

print("\nFiles in this directory:")
print(os.listdir("."))

Current working directory:
/home/users/joshua04/141

Files in this directory:
['Untitled3.ipynb', '.ipynb_checkpoints']


In [7]:
!cd ~/141
!mkdir -p data/raw data/processed src checkpoints results
!ls


Untitled3.ipynb  checkpoints  data  results  src


In [12]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
print(pd.__version__)

3.0.0


In [1]:
import pandas as pd
import numpy as np

print("pandas:", pd.__version__)
print("numpy:", np.__version__)


pandas: 3.0.0
numpy: 2.3.5


In [2]:
import os
import pandas as pd

DATA_ROOT = "/home/users/joshua04/141/data/raw/echonet_pediatric/A4C"

print("Exists:", os.path.exists(DATA_ROOT))
print("Contents:", os.listdir(DATA_ROOT))

filelist = pd.read_csv(os.path.join(DATA_ROOT, "FileList.csv"))
print(filelist.head())
print("Number of A4C studies:", len(filelist))


Exists: True
Contents: ['Videos', 'FileList.csv', 'FileList_new.csv', 'VolumeTracings_new.csv', 'VolumeTracings.csv']
                         FileName     EF Sex  Age  Weight  Height  Split
0  CR32a7555-CR32a7582-000039.avi  40.83   F    0    10.2    68.5      5
1  CR32a7555-CR32a97af-000033.avi  52.62   F    1    15.5    85.0      5
2  CR32a7555-CR32a97e1-000024.avi  24.85   F    0     4.0    56.0      5
3  CR32a7555-CR32a9850-000040.avi  50.96   F    4    18.0    99.0      5
4  CR32a7555-CR32a988d-000034.avi  56.76   F    0    13.2    75.0      5
Number of A4C studies: 3284


In [5]:
%pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 7.1 MB/s  0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 23.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 18.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [timm]2m14/15 [timm]ngface_hub]
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# ---------- video transform ----------
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def read_video(path, n_frames=16):
    cap = cv2.VideoCapture(path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    idxs = np.linspace(0, max(length - 1, 0), n_frames).astype(int)

    frames = []
    i = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if i in idxs:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(transform(frame))
        i += 1

    cap.release()

    while len(frames) < n_frames:
        frames.append(frames[-1])

    return torch.stack(frames, dim=1)  # (3, T, 224, 224)


class EchoNetA4C(Dataset):
    def __init__(self, root, split="train", n_frames=16):
        self.root = root
        self.n_frames = n_frames

        df = pd.read_csv(os.path.join(root, "FileList.csv"))
        df = df[df["Split"] != 5] if split == "train" else df[df["Split"] == 5]
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.root, "Videos", row["FileName"])
        x = read_video(path, self.n_frames)
        y = torch.tensor(row["EF"], dtype=torch.float32)
        return x, y


In [7]:
train_ds = EchoNetA4C(
    "/home/users/joshua04/141/data/raw/echonet_pediatric/A4C",
    split="train",
    n_frames=16
)

train_loader = DataLoader(
    train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

print("train_loader batches:", len(train_loader))


train_loader batches: 1466


In [8]:
import torch

model = torch.hub.load(
    "/home/users/joshua04/141/PanEcho",
    "PanEcho",
    source="local"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print("PanEcho ready on", device)


/home/users/joshua04/torch-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PanEcho ready on cuda


In [10]:
x, y = next(iter(train_loader))
x = x.to(device)

with torch.no_grad():
    out = model(x)

print(out.keys())


dict_keys(['pericardial-effusion', 'EF', 'GLS', 'LVEDV', 'LVESV', 'LVSV', 'LVSize', 'LVWallThickness-increased-any', 'LVWallThickness-increased-modsev', 'LVSystolicFunction', 'LVWallMotionAbnormalities', 'IVSd', 'LVPWd', 'LVIDs', 'LVIDd', 'LVOTDiam', 'LVDiastolicFunction', 'E|EAvg', 'RVSP', 'RVSize', 'RVSystolicFunction', 'RVIDd', 'TAPSE', 'RVSVel', 'LASize', 'LAIDs2D', 'LAVol', 'RASize', 'RADimensionM-L(cm)', 'AVStructure', 'AVStenosis', 'AVPkVel(m|s)', 'AVRegurg', 'LVOT20mmHg', 'MVStenosis', 'MVRegurgitation', 'TVRegurgitation', 'TVPkGrad', 'RAP-8-or-higher', 'AORoot'])


In [11]:
for k, v in out.items():
    print(k, v.shape)


pericardial-effusion torch.Size([2, 1])
EF torch.Size([2, 1])
GLS torch.Size([2, 1])
LVEDV torch.Size([2, 1])
LVESV torch.Size([2, 1])
LVSV torch.Size([2, 1])
LVSize torch.Size([2, 3])
LVWallThickness-increased-any torch.Size([2, 1])
LVWallThickness-increased-modsev torch.Size([2, 1])
LVSystolicFunction torch.Size([2, 3])
LVWallMotionAbnormalities torch.Size([2, 1])
IVSd torch.Size([2, 1])
LVPWd torch.Size([2, 1])
LVIDs torch.Size([2, 1])
LVIDd torch.Size([2, 1])
LVOTDiam torch.Size([2, 1])
LVDiastolicFunction torch.Size([2, 3])
E|EAvg torch.Size([2, 1])
RVSP torch.Size([2, 1])
RVSize torch.Size([2, 3])
RVSystolicFunction torch.Size([2, 1])
RVIDd torch.Size([2, 1])
TAPSE torch.Size([2, 1])
RVSVel torch.Size([2, 1])
LASize torch.Size([2, 3])
LAIDs2D torch.Size([2, 1])
LAVol torch.Size([2, 1])
RASize torch.Size([2, 1])
RADimensionM-L(cm) torch.Size([2, 1])
AVStructure torch.Size([2, 1])
AVStenosis torch.Size([2, 3])
AVPkVel(m|s) torch.Size([2, 1])
AVRegurg torch.Size([2, 3])
LVOT20mmHg t

In [12]:
ef_pred = out["EF"].squeeze().cpu()
ef_true = y[:len(ef_pred)]

for i in range(min(5, len(ef_pred))):
    print(f"Pred: {ef_pred[i].item():.2f} | True: {ef_true[i].item():.2f}")


Pred: 63.52 | True: 60.61
Pred: 59.32 | True: 52.92


In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

CUDA available: True
CUDA device count: 1
